
#Description:

* In this project, I will work with a dataset taken from Kaggle, where I will perform a credit risk analysis. The data provided comes from LendingClub, a financial services company based in San Francisco, California. LendingClub was the first peer-to-peer lending platform to register its offerings as securities with the Securities and Exchange Commission (SEC) and to offer loan trading on a secondary market. The company offers personal loans of up to $40,000, with terms ranging from 24 to 60 months, and operates entirely online, with no physical branches.

* In this project, I will apply the CRISP-DM method, which follows the following steps:

  * 1-**Business understanding**:

    What does the business need?

  * 2-**Data understanding**:

    What data do we have/need? Is it clean?

  * 3-**Data preparation**:

    How do we organize the data for modeling?

  * 4-**Modeling**:

    Which modeling techniques should we apply?

  * 5-**Evaluation**:

    What best meets the business objectives?

  * 6-**Deployment**:

    How do stakeholders access the results?

* **Categorical Variables**: 
  
  * For ordinal categorical variables, I will use the Ordinal Encoder. For nominal variables, the Target Encoder will be applied, with the aim of not increasing the dimensionality of our data, and also considering that this approach brings us more information for our training data because the variables are encoded according to their distributions in relation to the target variable, which in this case will be the **loan_status**.

* **Numerical Variables**: 
  
  * For numerical data, we will apply the normalization technique using MinMaxScaler. This choice aims to preserve the distribution of the data, considering that our distributions are almost all asymmetric, so opting for MinMaxScaler makes more sense in this context.

  * The decision to use MinMaxScaler also takes into account the fact that our data has a significant amount of outliers. However, these outliers are part of the natural distribution of the institution's data. Therefore, the objective of this choice is to allow the model to learn from this data.

* **Machine learning algorithms that will be used in this project**:

  * The basis of this project is a model with PyTorch, but first, to compare and verify the most suitable models for our data, a model will be created in each of the algorithms below:

  * Random Forest Classifier

  * KNN

  * Logistic Regression

  * XGBoost

  * Next, a network will be created in PyTorch. This network will undergo an initial training, in which we will evaluate the metrics and the results obtained.

* **How will the metrics for the evaluation stage be chosen?**:

  * The metrics that will be considered as a parameter to determine the best model and its effectiveness will be **AUC-ROC** together with **Accuracy**. Since we are dealing with binary classes and we have a minority class, which are the loans classified as unpaid, the accuracy of the model would not be enough to determine whether the model converged adequately with its classifications.

  * Therefore, I will be using the **AUC-ROC** metric as the first criterion as a main parameter to determine the effectiveness of the model. This metric takes into account the correct classification of the positive and negative classes, since in a dataset where we have an imbalance in the classes, it is common and natural to expect that the model has a tendency to classify most of the training and testing data with the majority class. Therefore, we will use this metric as a fundamental parameter for the evaluation and analysis of our models, so that we have a good control over false positives and false negatives, in order to reduce them as much as possible.

  * In the background, I will use the accuracy of the model to consider whether there is good predictability of the model with training, validation and testing data.

* **Hypertune | Finetune**:

  * After this initial training, we will **hypertune** the model, adjusting the hyperparameters and the number of neurons in the hidden layers.

  * Later, we will **finetune**, adjusting the learning level per epoch of the model.

* **Test data**

  * Finally, we will move on to the model evaluation stage, in which we will check its performance on the test data.

# Installs:

In [0]:
# install in the cluster > Libraries > Install new > PyPI  insert: >>>:

# threadpoolctl==3.5.0

# scikit-learn==1.5.2

# category_encoders==2.6.3

# torch==2.5.1

# torchmetrics==1.4.2

# xgboost==2.1.0

# ray[tune]==2.38.0

# Imports:

In [0]:
# Data modeling
# Pyspark.Api Pandas
from pyspark import pandas as ps

# Pandas
import pandas as pd

# Sklearning
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler,FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Target Encoder
from category_encoders import TargetEncoder


# Machine Learning

# Sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Sklearn Metrics
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay

# XGboost
import xgboost  as xgb

# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
from torch.utils.data import Dataset as Dataset
from torch.utils.data import WeightedRandomSampler

# Torch Metrics
import torchmetrics
from torchmetrics.classification import BinaryConfusionMatrix, BinaryAccuracy, BinaryF1Score, BinaryRecall, BinaryAUROC
from torchmetrics import MetricCollection
from torchmetrics import Accuracy, AUROC

# Graphics
# Matplotlib
import matplotlib.pyplot as plt # Graphics for metrics

import seaborn as sns

# Numpy
import numpy as np

# Python
import random
from functools import partial
import os
import tempfile
from pathlib import Path

# Ray tunner
from ray import tune
from ray import train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
import ray.cloudpickle as pickle

# Warnings
import warnings

# Options:

In [0]:
ps.set_option('display.max_rows', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore', category = FutureWarning)
#warnings.filterwarnings('ignore', category = UserWarning, module = 'threadpoolctl')

## Loading data train and data test

In [0]:
# File location and file type
file_location = 'dbfs:/FileStore/Data_Science_Credit_Risk_Analysis-Lending_Club/Datasets/Gold/train'
train_data = ps.read_parquet(file_location)

# File location and file type
file_location = 'dbfs:/FileStore/Data_Science_Credit_Risk_Analysis-Lending_Club/Datasets/Gold/test'
test_data = ps.read_parquet(file_location)

The train set

In [0]:
train_data.head()

,loan_amnt,term,int_rate,sub_grade,emp_length,home_ownership,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_util,total_acc,initial_list_status,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,mo_earliest_cr_line,real_income,ability_to_pay,expen_cr_inc,score_cr
0,5000.0,36 months,10.65,B2,10,RENT,Fully Paid,credit_card,27.65,A,B,3,A,83.7,9,f,A,79803.0,22100.0,323,17364.00,11.26,D,716.67
1,2500.0,60 months,15.27,C4,0,RENT,Charged Off,car,1.00,A,B,3,A,9.4,4,f,A,79803.0,22100.0,152,29700.00,2.42,B,875.00
2,2400.0,36 months,15.96,C5,10,RENT,Fully Paid,small_business,8.72,A,B,2,A,98.5,10,f,A,79803.0,22100.0,121,11183.63,9.05,C,708.33
3,10000.0,36 months,13.49,C1,10,RENT,Fully Paid,other,20.00,A,B,10,A,21.0,37,f,A,79803.0,22100.0,190,39360.00,10.34,C,858.33
4,5000.0,36 months,7.90,A4,3,RENT,Fully Paid,wedding,11.20,A,B,9,A,28.3,12,f,A,79803.0,22100.0,85,31968.00,5.87,C,733.33


Size trainset

In [0]:
train_data.shape

(195422, 24)

Info trainset

In [0]:
train_data.info()

<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 195422 entries, 0 to 195421
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   loan_amnt            195422 non-null  float64
 1   term                 195422 non-null  object 
 2   int_rate             195422 non-null  float64
 3   sub_grade            195422 non-null  object 
 4   emp_length           195422 non-null  object 
 5   home_ownership       195422 non-null  object 
 6   loan_status          195422 non-null  object 
 7   purpose              195422 non-null  object 
 8   dti                  195422 non-null  float64
 9   delinq_2yrs          195422 non-null  object 
 10  inq_last_6mths       195422 non-null  object 
 11  open_acc             195422 non-null  int32  
 12  pub_rec              195422 non-null  object 
 13  revol_util           195422 non-null  float64
 14  total_acc            195422 non-null  int32  
 15  initial_list_s

The test set

In [0]:
test_data.head()

,loan_amnt,term,int_rate,sub_grade,emp_length,home_ownership,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_util,total_acc,initial_list_status,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,mo_earliest_cr_line,real_income,ability_to_pay,expen_cr_inc,score_cr
0,9600.0,36 months,13.66,C3,10,RENT,Fully Paid,debt_consolidation,25.81,A,A,12,A,59.4,44,f,A,38566.0,27600.0,265,51191.10,7.65,C,791.67
1,7650.0,36 months,13.66,C3,0,RENT,Charged Off,debt_consolidation,34.81,A,B,11,A,91.9,20,f,A,64426.0,18300.0,148,32595.00,9.58,D,600.00
2,5250.0,36 months,11.44,B4,2,RENT,Fully Paid,debt_consolidation,14.36,A,A,8,A,20.2,13,f,A,10133.0,22000.0,67,22266.40,9.32,C,758.33
3,2500.0,36 months,11.99,B5,0,MORTGAGE,Fully Paid,home_improvement,13.77,A,B,9,A,94.3,32,f,A,397223.0,24300.0,158,76744.70,1.30,D,775.00
4,2000.0,36 months,14.99,C5,8,RENT,Fully Paid,credit_card,22.14,A,A,10,B,62.4,21,f,A,20539.0,15300.0,219,25070.92,3.32,D,733.33


Size test set

In [0]:
test_data.shape

(57509, 24)

Info test set

In [0]:
test_data.info()

<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 57509 entries, 0 to 57508
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   loan_amnt            57509 non-null  float64
 1   term                 57509 non-null  object 
 2   int_rate             57509 non-null  float64
 3   sub_grade            57509 non-null  object 
 4   emp_length           57509 non-null  object 
 5   home_ownership       57509 non-null  object 
 6   loan_status          57509 non-null  object 
 7   purpose              57509 non-null  object 
 8   dti                  57509 non-null  float64
 9   delinq_2yrs          57509 non-null  object 
 10  inq_last_6mths       57509 non-null  object 
 11  open_acc             57509 non-null  int32  
 12  pub_rec              57509 non-null  object 
 13  revol_util           57509 non-null  float64
 14  total_acc            57509 non-null  int32  
 15  initial_list_status  57509 non-nu

## Separating features and labels

#### Encoding the training and test data labels

In [0]:
train_data['loan_status'] = train_data['loan_status'].map({'Fully Paid': 0, 'Charged Off': 1}).astype(int)
test_data['loan_status'] = test_data['loan_status'].map({'Fully Paid': 0, 'Charged Off': 1}).astype(int)

In [0]:
train_data['loan_status'].head(10)

0    0
1    1
2    0
3    0
4    0
5    0
6    1
7    1
8    0
9    0
Name: loan_status, dtype: int64

In [0]:
test_data['loan_status'].head(10)

0    0
1    1
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: loan_status, dtype: int64

### Separating Features from Labels

In [0]:
# Train
X_train = train_data.drop(columns = ['loan_status']) 
y_train =  train_data['loan_status'].copy()

# Test
X_test = test_data.drop(columns = ['loan_status']) 
y_test =  test_data['loan_status'].copy()

In [0]:
print(f'The Train Features dataset shape: {X_train.shape}')
print(f'The Train Labels dataset shape: {y_train.shape}')
print(f'The Test Features dataset shape: {X_test.shape}')
print(f'The Test Labels dataset shape: {y_test.shape}')

The Train Features dataset shape: (195422, 23)
The Train Labels dataset shape: (195422,)
The Test Features dataset shape: (57509, 23)
The Test Labels dataset shape: (57509,)


Adjusting the datasets to pd.dataframe type

In [0]:
X_train = X_train.to_pandas()
y_train = y_train.to_pandas()
X_test = X_test.to_pandas()
y_test = y_test.to_pandas()

### Checking numeric columns

In [0]:
numerical_continuos = X_train.select_dtypes('float64').columns.to_list()
numerical_discrete = X_train.select_dtypes('int32').columns.to_list()
numerical_features = numerical_continuos
numerical_features.extend(numerical_discrete)
print(f'There are {len(numerical_features)} numerical features.')
print('\nThere are:')
numerical_features

There are 12 numerical features.

There are:


['loan_amnt',
 'int_rate',
 'dti',
 'revol_util',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'real_income',
 'ability_to_pay',
 'score_cr',
 'open_acc',
 'total_acc',
 'mo_earliest_cr_line']

### Checking categorical columns

In [0]:
categorical_features = X_train.select_dtypes('object').columns.to_list()
print(f'There are {len(categorical_features)} categorical features.')
print('\nThere are:')
categorical_features

There are 11 categorical features.

There are:


['term',
 'sub_grade',
 'emp_length',
 'home_ownership',
 'purpose',
 'delinq_2yrs',
 'inq_last_6mths',
 'pub_rec',
 'initial_list_status',
 'tot_coll_amt',
 'expen_cr_inc']

In [0]:
for feature in categorical_features:
  print(feature)
  print('-' * 40)
  print(f'There are {X_train[feature].nunique()} unique values there are: ')
  print(X_train[feature].value_counts(normalize = True))
  print()

term
----------------------------------------
There are 2 unique values there are: 
 36 months    0.794957
 60 months    0.205043
Name: term, dtype: float64

sub_grade
----------------------------------------
There are 35 unique values there are: 
B3    0.078645
B4    0.071282
B2    0.062163
C1    0.059077
B5    0.057522
C2    0.055690
B1    0.050593
A5    0.049104
C3    0.048342
A4    0.046960
C4    0.044928
C5    0.042165
D1    0.038005
D2    0.034986
A3    0.030769
D3    0.029792
D4    0.027366
A2    0.025662
A1    0.024004
D5    0.022500
E2    0.016170
E1    0.015674
E3    0.013131
E4    0.011682
E5    0.009738
F1    0.007921
F2    0.006427
F3    0.005593
F4    0.004319
F5    0.003352
G1    0.002374
G2    0.001637
G3    0.001095
G4    0.000737
G5    0.000594
Name: sub_grade, dtype: float64

emp_length
----------------------------------------
There are 11 unique values there are: 
10    0.294583
0     0.119951
2     0.094089
3     0.081726
5     0.076255
1     0.066799
4     0.06569

### Checking labels

In [0]:
  print('loan_status')
  print('-' * 40)
  print(f'There are {y_train.nunique()} unique values there are: ')
  print(y_train.value_counts(normalize = True))
  print()

loan_status
----------------------------------------
There are 2 unique values there are: 
0    0.814698
1    0.185302
Name: loan_status, dtype: float64



## Preprocessing

### Categorical Features

In [0]:
# Ordinal features
ordinal_features = [
     'term', 'sub_grade', 'expen_cr_inc'  
]
# Manual adjustment
ordinal_emp_length = ['emp_length']

# Nominal Features

nominal_features = ['home_ownership', 'purpose', 'initial_list_status', 'tot_coll_amt', 'delinq_2yrs', 'pub_rec', 'inq_last_6mths',]


### Numerical Features

In [0]:

num_features = [
  'loan_amnt', 'int_rate', 'dti', 'open_acc', 'revol_util', 'total_acc','tot_cur_bal', 'total_rev_hi_lim', 'real_income', 'ability_to_pay', 'score_cr', 'mo_earliest_cr_line',
]

In [0]:
# Categorical ordinal
categorical_ordinal = Pipeline(
    steps = [
        ('ordinal_encoder', OrdinalEncoder()),
        ('min_max_scaler', MinMaxScaler()),
    ]
)
# emp_length
emp_length_ordinal = Pipeline(
    steps = [
        ('ordinal_encoder', OrdinalEncoder(categories = [['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']])),
        ('min_max_scaler', MinMaxScaler()),
    ]
)


# Categorical Nominal
categorical_nominal = Pipeline(
    steps = [
        ('target_encoder', TargetEncoder(cols = nominal_features)),
        ('min_max_scaler', MinMaxScaler()),
    ]
)


# Column Transformer
preprocessor = ColumnTransformer(
    transformers = [
        ('ordinal', categorical_ordinal, ordinal_features),
        ('ord_emp_length', emp_length_ordinal, ordinal_emp_length),
        ('target', categorical_nominal, nominal_features),
        ('numerical_features', MinMaxScaler(), num_features),
    ],
    remainder = 'passthrough'
)


In [0]:
X_train_preprocessed = preprocessor.fit_transform(X_train, y_train)

In [0]:
X_train_preprocessed[0]

array([0.        , 0.17647059, 1.        , 1.        , 1.        ,
       0.21474779, 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.13043478, 0.25339147, 0.79022578, 0.05769231,
       0.54705882, 0.07216495, 0.01672125, 0.03137867, 0.01486159,
       0.33412817, 0.46875293, 0.37912814])

### HyperTunning

#### Dataset Class Pytoch

In [0]:
class Dataset(Dataset):

    # Initializing with separate data and labels
    def __init__(self, X_data, y_data):

        # Data and Labels
        self.X_data = X_data
        self.y_data = y_data
    
    # Get item
    def __getitem__(self, idx):

        # Getting the feature and labels
        sample = self.X_data[idx]
        label = self.y_data[idx]

        # Transforming to tensor
        sample = torch.from_numpy(sample.astype(np.float32))
        label = torch.from_numpy(label.astype(np.float32))
        return sample, label
    
    # length od dataset
    def __len__(self):
        return len(self.X_data)

#### Adjusting dimensions of training data

In [0]:
# Train

# Transforming labels to a dataset in numpy
y_train_numpy = y_train.to_numpy() 

# Adjusting label dimensions to 2D
y_train_numpy = y_train_numpy[:, np.newaxis]

train_set = Dataset(X_data = X_train_preprocessed, y_data = y_train_numpy)


Checking the data converted to tensors

In [0]:
data, label = train_set[0]
data, label

(tensor([0.0000, 0.1765, 1.0000, 1.0000, 1.0000, 0.2147, 0.0000, 0.0000, 0.0000,
         0.0000, 1.0000, 0.1304, 0.2534, 0.7902, 0.0577, 0.5471, 0.0722, 0.0167,
         0.0314, 0.0149, 0.3341, 0.4688, 0.3791]),
 tensor([0.]))

#### Checking the size of input and output data

In [0]:
for X, y in train_set:
    print(f'Shape of X_train: {X.shape} {X.dtype}')
    print(f'Shape of y_train: {y.shape} {y.dtype}')
    break

Shape of X_train: torch.Size([23]) torch.float32
Shape of y_train: torch.Size([1]) torch.float32


### Data loaders

* We wrap the data loaders in their own function and pass a global data directory. This way we can share a data directory between different trials.

In [0]:
def load_data(data_dir = './'):

    # Loading Train set
    trainset = Dataset(X_data = X_train_preprocessed, y_data = y_train_numpy)

    # Spliting trainset and validationset
    train_size = int(len(trainset) * 0.8)
    val_size = int(len(trainset) - train_size)


    train_subset, val_subset = random_split(
        trainset, [train_size, val_size]
    )
    
    return train_subset, val_subset

### Configurable neural network

In [0]:
class Net_Hyper(nn.Module):

  # Initialize weights
  def initialize_weights(self):

    for layer in self.features:

      if isinstance(layer, nn.Linear):
        torch.nn.init.kaiming_normal_(layer.weight, nonlinearity = 'relu')
        torch.nn.init.constant_(layer.bias, 0)

    for layer in self.classifier:
      
      if isinstance(layer, nn.Linear):
        torch.nn.init.kaiming_normal_(layer.weight, nonlinearity = 'relu')
        torch.nn.init.constant_(layer.bias, 0)

  # Net Layers
  def __init__(self, l1 = 64, l2 = 64 , l3 = 64, drop_p = 0.10, input_layer = len(train_set[0][0]), output_layer = 1, ):
    super(Net_Hyper, self).__init__()
    
    # Features
    self.features = nn.Sequential(

      # Input | Layer 1
      nn.Linear(input_layer, l1),
      nn.BatchNorm1d(l1),
      nn.ReLU(),
      nn.Dropout(p = drop_p),

      # Layer 2
      nn.Linear(l1, l2),
      nn.BatchNorm1d(l2),
      nn.ReLU(),
      nn.Dropout(p = drop_p),

      # Layer 3
      nn.Linear(l2, l3),
      nn.BatchNorm1d(l3),
      nn.ReLU(),
      #nn.Dropout(p = drop_p),

    )

    # Classifier
    self.classifier = nn.Sequential(
      # Output Layer
      nn.Linear(l3, output_layer)
    )

    # Initialize weights
    self.initialize_weights()

  # Forward
  def forward(self, X):
    
    X = self.features(X)
    logits = self.classifier(X)
    return logits
  

In [0]:
net = Net_Hyper()
print(net)

Net_Hyper(
  (features): Sequential(
    (0): Linear(in_features=23, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=64, out_features=64, bias=True)
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=64, out_features=1, bias=True)
  )
)


### The train function

In [0]:
def train_hyper(config, data_dir = None):
    
    # Net 
    net = Net_Hyper(config['l1'], config['l2'], config['l3'])

    # Cheking device
    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda:0'
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)
    
    # Criterion | Optimizer
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(net.parameters(), lr = config['lr'], weight_decay = 5e-4)


    # Check Point
    checkpoint = get_checkpoint()
    if checkpoint:
        with checkpoint.as_directory() as checkpoint_dir:
            data_path = Path(checkpoint_dir) / 'data.pkl'
            with open(data_path, 'rb') as fp:
                checkpoint_state = pickle.load(fp)
            start_epoch = checkpoint_state['epoch']
            net.load_state_dict(checkpoint_state['net_state_dict'])
            optimizer.load_state_dict(checkpoint_state['optimizer_state_dict'])
    else:
        start_epoch = 0

    # Load train_set and val_set
    trainset, valset = load_data(data_dir)

    # Applying the sampler function to the training data for class balancing
    # Distributing the targets
    targets = torch.cat([labels for _, labels in trainset])
    targets = targets.to(torch.int64)
    class_counts = torch.bincount(targets)
    class_weights = 1.0 / class_counts.float()
    sample_weights = class_weights[targets]
    sampler =  WeightedRandomSampler(weights = sample_weights, num_samples = len(sample_weights), replacement = True)

    # DataLoader
    # Train loader
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size = int(config['batch_size']), sampler = sampler, num_workers = 2, drop_last= True,
    )
    # Val loader
    valloader = torch.utils.data.DataLoader(
        valset, batch_size = int(config['batch_size']), shuffle = True, num_workers = 2, drop_last= False,
    )

    # Loop over the dataset multiple times
    for epoch in range(start_epoch, 50):

        # Training
        net.train()

        # Metrics training
        running_loss = 0.0 
        epoch_steps = 0

        for i, data in enumerate(trainloader, 0):

            # Get the inputs; data is a list of [inputs, labels]
            inputs, labels = data # X and y
            inputs, labels = inputs.to(device), labels.to(device)

            # Foward Pass
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Backward + optimize
            loss.backward()
            optimizer.step()

            # Print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999: #print every 2000 mini-batches
                print(
                    '[%d, %5d] loss: %.3f'
                    % (epoch + 1, i + 1, running_loss / epoch_steps)
                )
                running_loss = 0.0
        
        # Validation
        # Metrics for epochs
        avg_loss, avg_accuracy, avg_auc = [], [], []


        # Initialize AUROC and Accuracy for binary classification
        accuracy_val = Accuracy(task = 'binary').to(device)
        auc_val = torchmetrics.AUROC(task = 'binary').to(device)

        # Metrics validation
        val_loss = 0.0
        val_steps = 0

        all_preds = []
        all_labels = []

        # Net eval
        net.eval()
        for i, data in enumerate(valloader, 0):

            with torch.no_grad():
                
                # Get the inputs; data is a list of [inputs, labels]
                inputs, labels = data
                inputs, labels =  inputs.to(device), labels.to(device)

                # Eval net
                outputs = net(inputs)
                loss = criterion(outputs, labels)

                # Accumulating Predictions
                all_preds.append(torch.sigmoid(outputs).cpu())
                all_labels.append(labels.cpu())
                
                # Accumulating Loss
                val_loss += loss.item()
                val_steps += 1

        # Calculate AUC-ROC and Accuracy for validation
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)

        # Accuracy
        avg_accuracy.append(accuracy_val(all_preds, all_labels.int()).item()) 

        # AUC
        avg_auc.append(auc_val(all_preds, all_labels.int()).item())

        # Loss
        avg_loss.append(val_loss / val_steps)

        # Checkpoint
        checkpoint_data = {
            'epoch': epoch,
            'net_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }
        with tempfile.TemporaryDirectory() as checkpoint_dir:
            data_path = Path(checkpoint_dir) / 'data.pkl'
            with open(data_path, 'wb') as fp:
                pickle.dump(checkpoint_data, fp)

            checkpoint = Checkpoint.from_directory(checkpoint_dir)
            train.report(
                {'loss': avg_loss[-1], 'accuracy': avg_accuracy[-1], 'auc_roc': avg_auc[-1]},
                checkpoint = checkpoint,
            )
    
    print('Finished Training')

### Test set accuracy

In [0]:
def test_hyper(net, device = 'cpu'):

    # Loading Dataset
    train_set, valset = load_data()

    # DataLoader
    testloader = torch.utils.data.DataLoader(
        valset, batch_size = 4, shuffle = False, num_workers = 2
    )

    # Criterion
    criterion = nn.BCEWithLogitsLoss()

    # Test
    # Metrics for epochs
    avg_loss, avg_accuracy, avg_auc = [], [], []


    # Initialize AUROC and Accuracy for binary classification
    accuracy_test = Accuracy(task = 'binary').to(device)
    auc_test = torchmetrics.AUROC(task = 'binary').to(device)

    # Metrics validation
    test_loss = 0.0
    test_steps = 0

    all_preds = []
    all_labels = []

    # Net eval
    net.eval()
    for i, data in enumerate(testloader, 0):

        with torch.no_grad():

            inputs, labels = data
            inputs, labels =  inputs.to(device), labels.to(device)

            # Eval net
            outputs = net(inputs)
            loss = criterion(outputs, labels)

            # Accumulating Predictions
            all_preds.append(torch.sigmoid(outputs).cpu())
            all_labels.append(labels.cpu())

            # Accumulating Loss
            test_loss += loss.item()
            test_steps += 1

    # Calculate AUC-ROC and Accuracy for validation
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # Accuracy
    avg_accuracy.append(accuracy_test(all_preds, all_labels.int()).item()) 

    # AUC
    avg_auc.append(auc_test(all_preds, all_labels.int()).item())

    # Loss
    avg_loss.append(test_loss / test_steps)

    return avg_loss, avg_accuracy, avg_auc

### Configuring the search space

In [0]:
def main(num_samples = 50, max_num_epochs = 50, gpus_per_trial = 2):
    
    # Setting directory
    data_dir = os.path.abspath('./')
    load_data(data_dir)

    # Config hyperparameters
    config = {
        'l1': tune.choice([2 ** i for i in range(9)]), 
        'l2': tune.choice([2 ** i for i in range(9)]),
        'l3': tune.choice([2 ** i for i in range(9)]),
        'lr': tune.loguniform(1e-5, 1e-3),
        'batch_size': tune.choice([128, 256, 512]),
    }


    # Adjusting metric to be minimized
    scheduler = ASHAScheduler(
        metric = 'auc_roc',
        mode = 'max', 
        max_t = max_num_epochs, 
        grace_period = 5,
        reduction_factor = 2,
    )

    # Result
    result = tune.run(
        partial(train_hyper, data_dir = data_dir),
        resources_per_trial = {'cpu': 2, 'gpu': gpus_per_trial},
        config = config,
        num_samples =  num_samples, 
        scheduler = scheduler,
    )


    # Printing results
    best_trial = result.get_best_trial('loss', 'min', 'last')
    print(f"Best trial config: {best_trial.config}")
    print(f"\nBest trial final validation loss: {best_trial.last_result['loss']}")
    print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")
    print(f"Best trial final validation auc_roc: {best_trial.last_result['auc_roc']}")

    best_trained_model = Net_Hyper(best_trial.config['l1'], best_trial.config['l2'], best_trial.config['l3'])
    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda:0'
        if gpus_per_trial > 1:
            best_trained_model == nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    # BestCheckpoint
    best_checkpoint = result.get_best_checkpoint(trial = best_trial, metric = 'loss', mode = 'min')
    with best_checkpoint.as_directory() as checkpoint_dir:
        data_path = Path(checkpoint_dir) / 'data.pkl'
        with open(data_path, 'rb') as fp:
            best_checkpoint_data = pickle.load(fp)
    
    best_trained_model.load_state_dict(best_checkpoint_data['net_state_dict'])
    test_loss, test_acc, test_roc = test_hyper(best_trained_model, device)
    print(f'\nBest trial test set loss: {test_loss}')
    print(f'Best trial test set accuracy: {test_acc}')
    print(f'Best trial test set auc_roc:{test_roc}')

# Main

if __name__  == '__main__':
    main(num_samples = 30, max_num_epochs = 50, gpus_per_trial = 0)

2024-11-28 08:19:31,465	INFO worker.py:1816 -- Started a local Ray instance.
2024-11-28 08:19:33,378	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.


+--------------------------------------------------------------------+
| Configuration for experiment     train_hyper_2024-11-28_08-19-33   |
+--------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator             |
| Scheduler                        AsyncHyperBandScheduler           |
| Number of trials                 30                                |
+--------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_hyper_2024-11-28_08-19-33
To visualize your results with TensorBoard, run: `tensorboard --logdir /local_disk0/tmp/ray/session_2024-11-28_08-19-28_875118_6646/artifacts/2024-11-28_08-19-33/train_hyper_2024-11-28_08-19-33/driver_artifacts`
(raylet) Warning: The actor ImplicitFunc is very large (71 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray ob

(func pid=7839) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_hyper_2024-11-28_08-19-33/train_hyper_7fd20_00000_0_batch_size=256,l1=2,l2=4,l3=2,lr=0.0000_2024-11-28_08-19-49/checkpoint_000000)



Trial train_hyper_7fd20_00000 finished iteration 2 at 2024-11-28 08:20:22. Total running time: 47s
+------------------------------------------------------------+
| Trial train_hyper_7fd20_00000 result                       |
+------------------------------------------------------------+
| checkpoint_dir_name                      checkpoint_000001 |
| time_this_iter_s                                   6.26976 |
| time_total_s                                      15.12947 |
| training_iteration                                       2 |
| accuracy                                           0.44828 |
| auc_roc                                            0.52236 |
| loss                                               0.68967 |
+------------------------------------------------------------+
Trial train_hyper_7fd20_00000 saved a checkpoint for iteration 2 at: (local)/root/ray_results/train_hyper_2024-11-28_08-19-33/train_hyper_7fd20_00000_0_batch_size=256,l1=2,l2=4,l3=2,lr=0.0000_2024-11-28_08-1

(func pid=7839) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_hyper_2024-11-28_08-19-33/train_hyper_7fd20_00000_0_batch_size=256,l1=2,l2=4,l3=2,lr=0.0000_2024-11-28_08-19-49/checkpoint_000001)


...


Trial train_hyper_7fd20_00029 finished iteration 3 at 2024-11-28 10:07:02. Total running time: 1hr 47min 27s
+------------------------------------------------------------+
| Trial train_hyper_7fd20_00029 result                       |
+------------------------------------------------------------+
| checkpoint_dir_name                      checkpoint_000002 |
| time_this_iter_s                                   9.85389 |
| time_total_s                                      32.84183 |
| training_iteration                                       3 |
| accuracy                                           0.71529 |
| auc_roc                                            0.64802 |
| loss                                                0.6439 |
+------------------------------------------------------------+
Trial train_hyper_7fd20_00029 saved a checkpoint for iteration 3 at: (local)/root/ray_results/train_hyper_2024-11-28_08-19-33/train_hyper_7fd20_00029_29_batch_size=128,l1=64,l2=1,l3=256,lr=0.0000_2

(func pid=61471) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_hyper_2024-11-28_08-19-33/train_hyper_7fd20_00029_29_batch_size=128,l1=64,l2=1,l3=256,lr=0.0000_2024-11-28_08-19-58/checkpoint_000002)



Trial status: 29 TERMINATED | 1 RUNNING
Current time: 2024-11-28 10:07:12. Total running time: 1hr 47min 37s
Logical resource usage: 2.0/2 CPUs, 0/0 GPUs
+--------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                status         l1     l2     l3            lr     batch_size     iter     total time (s)       loss     accuracy     auc_roc |
+--------------------------------------------------------------------------------------------------------------------------------------------------------+
| train_hyper_7fd20_00029   RUNNING        64      1    256   3.47205e-05            128        3            32.8418   0.643896     0.715287    0.648017 |
| train_hyper_7fd20_00000   TERMINATED      2      4      2   1.95684e-05            256       50           328.572    0.653298     0.700473    0.655163 |
| train_hyper_7fd20_00001   TERMINATED      4    256      8   2.1644e-

(func pid=61471) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_hyper_2024-11-28_08-19-33/train_hyper_7fd20_00029_29_batch_size=128,l1=64,l2=1,l3=256,lr=0.0000_2024-11-28_08-19-58/checkpoint_000003)
2024-11-28 10:07:23,057	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_hyper_2024-11-28_08-19-33' in 0.0479s.
(func pid=61471) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_hyper_2024-11-28_08-19-33/train_hyper_7fd20_00029_29_batch_size=128,l1=64,l2=1,l3=256,lr=0.0000_2024-11-28_08-19-58/checkpoint_000004)



Trial train_hyper_7fd20_00029 finished iteration 5 at 2024-11-28 10:07:22. Total running time: 1hr 47min 48s
+------------------------------------------------------------+
| Trial train_hyper_7fd20_00029 result                       |
+------------------------------------------------------------+
| checkpoint_dir_name                      checkpoint_000004 |
| time_this_iter_s                                   9.99241 |
| time_total_s                                      53.06928 |
| training_iteration                                       5 |
| accuracy                                           0.70915 |
| auc_roc                                            0.66029 |
| loss                                                0.6451 |
+------------------------------------------------------------+
Trial train_hyper_7fd20_00029 saved a checkpoint for iteration 5 at: (local)/root/ray_results/train_hyper_2024-11-28_08-19-33/train_hyper_7fd20_00029_29_batch_size=128,l1=64,l2=1,l3=256,lr=0.0000_2